# CUDA-Q

## What is the Computational Configuration of the Supercomputing?

In [ ]:
!nvidia-smi #topo -m 

## How to start the Anaconda Environment CUDA-Q on Kuatomu?

### `using python`

```cpp
Steps:

1) ssh kuatomu
2) module load anaconda3/4.13.0
3) source activate cudaq-env
4) unset PYTHONPATH
```

### `using C++`

```cpp
Steps:

1) ssh kuatomu
2) module load anaconda3/4.13.0
3) source activate cudaq-env
4) unset PYTHONPATH
5) source /opt/nvidia/cudaq/set_env.sh
```

## Sample test CUDA-Q on Kuatomu

### `python`

In [ ]:
%%writefile sample-cudaq.py
import cudaq

qubit_count = 2

print(cudaq.draw(kernel, qubit_count))

results = cudaq.sample(kernel, qubit_count)
# Should see a roughly 50/50 distribution between the |00> and
# |11> states. Example: {00: 505  11: 495}

print("Measurement distribution:" + str(results))

In [ ]:
!python sample-cudaq.py

## `C++`

In [ ]:
%%writefile sample-cudaq.cpp
#include <cudaq.h>

struct bell {
  int operator()(int num_iters) __qpu__ {
    cudaq::qarray<2> q;
    int nCorrect = 0;
    for (int i = 0; i < num_iters; i++) {
      h(q[0]);
      x<cudaq::ctrl>(q[0], q[1]);
      auto results = mz(q);
      if (results[0] == results[1])
        nCorrect++;

      reset(q[0]);
      reset(q[1]);
    }
    return nCorrect;
  }
};

int main(int argc, char **argv) { 
    
  printf("N Correct = %d\n", bell{}(100)); 
 
  return 0;
   
}

In [ ]:
!nvq++ sample-cudaq.cpp -o sample-cudaq

In [ ]:
!./sample-cudaq